In [1]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [21]:
es.cluster.health()

{'active_primary_shards': 15,
 'active_shards': 15,
 'active_shards_percent_as_number': 100.0,
 'cluster_name': 'smartpub',
 'delayed_unassigned_shards': 0,
 'initializing_shards': 0,
 'number_of_data_nodes': 1,
 'number_of_in_flight_fetch': 0,
 'number_of_nodes': 1,
 'number_of_pending_tasks': 0,
 'relocating_shards': 0,
 'status': 'green',
 'task_max_waiting_in_queue_millis': 0,
 'timed_out': False,
 'unassigned_shards': 0}

In [89]:
es.nodes.stats()

{'_nodes': {'failed': 0, 'successful': 1, 'total': 1},
 'cluster_name': 'smartpub',
 'nodes': {'ebADX0twTlqxi_hYrNewow': {'adaptive_selection': {'ebADX0twTlqxi_hYrNewow': {'avg_queue_size': 0,
     'avg_response_time_ns': 120065,
     'avg_service_time_ns': 64326,
     'outgoing_searches': 0,
     'rank': '0.1'}},
   'breakers': {'accounting': {'estimated_size': '1.8mb',
     'estimated_size_in_bytes': 1888034,
     'limit_size': '990.7mb',
     'limit_size_in_bytes': 1038876672,
     'overhead': 1.0,
     'tripped': 0},
    'fielddata': {'estimated_size': '0b',
     'estimated_size_in_bytes': 0,
     'limit_size': '594.4mb',
     'limit_size_in_bytes': 623326003,
     'overhead': 1.03,
     'tripped': 0},
    'in_flight_requests': {'estimated_size': '78.8kb',
     'estimated_size_in_bytes': 80692,
     'limit_size': '990.7mb',
     'limit_size_in_bytes': 1038876672,
     'overhead': 1.0,
     'tripped': 0},
    'parent': {'estimated_size': '1.8mb',
     'estimated_size_in_bytes': 1968

# 2 Full Index

In [4]:
from pymongo import MongoClient
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import nltk
import config as cfg

sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

In [5]:
client = MongoClient('localhost:' + str(cfg.mongoDB_Port))
db = client.pub
pub = client.pub.publications
es = Elasticsearch(
    [{'host': 'localhost', 'port': 9200}], timeout=30, max_retries=10, retry_on_timeout=True
)
es.cluster.health(wait_for_status='yellow', request_timeout=1)

{'active_primary_shards': 0,
 'active_shards': 0,
 'active_shards_percent_as_number': 100.0,
 'cluster_name': 'smartpub',
 'delayed_unassigned_shards': 0,
 'initializing_shards': 0,
 'number_of_data_nodes': 1,
 'number_of_in_flight_fetch': 0,
 'number_of_nodes': 1,
 'number_of_pending_tasks': 0,
 'relocating_shards': 0,
 'status': 'green',
 'task_max_waiting_in_queue_millis': 0,
 'timed_out': False,
 'unassigned_shards': 0}

In [61]:
def return_chapters(mongo_string_search, db):
    results = db.publications.find(mongo_string_search)
    chapters = list()
    chapter_nums = list()
    list_of_docs = list()
    merged_chapters = list()
    
    my_dict = {
        "_id": "",
        "title": "",
        "content": "",
        "journal": "",
        "year":""
    }
    for i, r in enumerate(results):
        # try:
        # list_of_sections = list()
        my_dict['_id'] = r['_id']
        my_dict['title'] = r['title']
        try:
            my_dict['journal'] = r['booktitle']
        except: 
            pass
        try:
            my_dict['journal'] = r['journal']
        except: 
            pass
        try:
            my_dict['year'] = r['year']
        except: 
            pass
        try:
            my_dict['content'] = r['content']['fulltext']
        except:
            my_dict['content'] = ""
            # print(my_dict)
            # sys.exit(1)

        list_of_docs.append(my_dict)

        my_dict = {
            "_id": "",
            "title": "",
            "content": "",
            "journal": "",
            "year": ""
        }

    return list_of_docs

In [62]:
filter_conference = ["WWW", "ICSE", "VLDB", "JCDL", "TREC",  "SIGIR", "ICWSM", "ECDL", "ESWC", "TPDL", 
                     'pbio', 'pgen', 'bmcgen', 'bmcbiot', 'bmcneur', 'bmceb', 'genbio', 'bcr']

list_of_pubs = []

for booktitle in filter_conference:
    mongo_string_search = {'$or': [{'$and': [{'booktitle': booktitle}, {'content.fulltext': {'$exists': True}}]} ,
                                   {'$and': [{'journal': booktitle},   {'content.fulltext': {'$exists': True}}]} ]}
    list_of_pubs.append(return_chapters(mongo_string_search, db))

In [64]:
len(list_of_pubs)

18

In [ ]:
for pubs in list_of_pubs:
    actions = []
    
    for cur in pubs:
        text = cur["content"]
        
        print(cur['_id'])
        print(cur['journal'])

        actions.append({
                    "_index": "ir",
                    "_type": "publications",
                    "_id" : cur['_id'],
                    "journal": cur['journal'],
                    "year": cur['year'],
                    "_source" : {
                        "text" : text,
                        "title": cur["title"]
                    }
                })

    if len(actions) == 0:
            continue

    res = helpers.bulk(es, actions)
    print(res)

# 3 Twosent Index

In [29]:
from pymongo import MongoClient
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import math
import requests
import nltk
import _pickle as cPickle
import config as cfg
import logging

###############################
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
client = MongoClient('localhost:' + str(cfg.mongoDB_Port))
pub = client.pub.publications
db = client.pub
es = Elasticsearch([{'host': 'localhost', 'port': 9200}],
                   timeout=30, max_retries=10, retry_on_timeout=True)
es.cluster.health(wait_for_status='yellow', request_timeout=1)

{'active_primary_shards': 5,
 'active_shards': 5,
 'active_shards_percent_as_number': 100.0,
 'cluster_name': 'smartpub',
 'delayed_unassigned_shards': 0,
 'initializing_shards': 0,
 'number_of_data_nodes': 1,
 'number_of_in_flight_fetch': 0,
 'number_of_nodes': 1,
 'number_of_pending_tasks': 0,
 'relocating_shards': 0,
 'status': 'green',
 'task_max_waiting_in_queue_millis': 0,
 'timed_out': False,
 'unassigned_shards': 0}

In [30]:
def return_paragraphs(mongo_string_search, db):
    # mongo_string_search = {"dblpkey": "{}".format(dblkey)}
    results = db.publications.find(mongo_string_search)
    chapters = list()
    chapter_nums = list()
    list_of_docs = list()
    # list_of_abstracts = list()
    merged_chapters = list()
    
    my_dict = {
        "_id": "",
        "paragraphs": list(),
        "title": ""
    }
    
    for i, r in enumerate(results):
        # try:
        # list_of_sections = list()
        my_dict['_id'] = r['_id']
        my_dict['title'] = r['title']
        paragraphs = []
        
#         ########################################################
        try:
            for chapter in r['content']['chapters']:
                if (chapter == {}):
                    continue

                    # remove the filter that removes related works
                    # elif str(chapter['title']).lower() in filter_chapters:
                    # print(chapter['title'])

                # print(chapter['title'])
                for paragraph in chapter['paragraphs']:
                    if paragraph == {}:
                        continue
                    paragraphs.append(paragraph)

            my_dict['paragraphs'] = paragraphs

        except:
            logging.exception('No chapters in ' + r['_id'], exc_info=True)
            
                
#         try:
#             for paragraph in r['content']['paragraphs']:
#                 if paragraph == {}:
#                     continue
#                 paragraphs.append(paragraph['text'])
#             my_dict['paragraphs'] = paragraphs
        
#         except:
#             logging.exception('No paragraphs in ' + r['_id'], exc_info=True)
#             continue
#         ########################################################    

        list_of_docs.append(my_dict)
        my_dict = {
            "_id": "",
            "paragraphs": list(),
            "title": ""
        }

    return list_of_docs

In [38]:
results = db.publications.find(mongo_string_search)
mongo_string_search = {'$or': [{'$and': [{'booktitle': publication}, {'content.fulltext': {'$exists': True}}]} ,
                                   {'$and': [{'journal': publication},   {'content.fulltext': {'$exists': True}}]} ]}

In [39]:
my_dict = {
    "_id": "",
    "paragraphs": list(),
    "title": ""
}
for i, r in enumerate(results):
    # try:
    # list_of_sections = list()
    my_dict['_id'] = r['_id']
    my_dict['title'] = r['title']
    paragraphs = []

In [60]:
for chapter in r['content']['chapters']:
    for paragraph in chapter[0]['paragraphs']:
        print(paragraph)
        print('')

Eukaryotic cells utilize intricate mechanisms for the uptake and intracellular sorting of various macromolecules, such as membrane components and extracellular proteins. Microscopic imaging studies have helped considerably to describe and define important steps of the uptake process. It has been shown that the cytosolic face of the plasma membrane is studded with small, domed assemblages of peripheral membrane proteins. These constitute transitory sorting stations that dynamically remodel the composition of the cell surface in response to both intracellular and extracellular stimuli. Typically, within a minute of forming, these assemblages invaginate to generate “buds” that then detach, generating small (∼60 nm), membrane-bound transport vesicles that deliver their contents (often receptors) to specific intracellular compartments—namely acceptor early endosomes—for further dissemination within the cell (Figure 1). This invagination and sorting process is called endocytosis, and althoug

In [12]:
# filter_publications = ["WWW", "ICSE", "VLDB", "JCDL", "TREC",  "SIGIR", "ICWSM", "ECDL", "ESWC", "TPDL", 
filter_publications = [ "PLoS Biology"]#, "Breast Cancer Research", "BMC Evolutionary Biology", "BMC Genomics", "BMC Biotechnology",
#                         "BMC Neuroscience", "Genome Biology", "PLoS Genetics", "Breast Cancer Research : BCR", 
#                        "Genome Biology and Evolution", "Breast Cancer Research and Treatment"]

list_of_pubs = []

for publication in filter_publications:
    mongo_string_search = {'$or': [{'$and': [{'booktitle': publication}, {'content.fulltext': {'$exists': True}}]} ,
                                   {'$and': [{'journal': publication},   {'content.fulltext': {'$exists': True}}]} ]}
    list_of_pubs.append(return_paragraphs(mongo_string_search, db))

print("Total journals:", len(list_of_pubs))


ERROR:root:No chapters in PMC_3472976
Traceback (most recent call last):
  File "<ipython-input-11-ddacd84d4e65>", line 34, in return_paragraphs
    for paragraph in chapter['paragraphs']:
TypeError: string indices must be integers
ERROR:root:No chapters in PMC_521730
Traceback (most recent call last):
  File "<ipython-input-11-ddacd84d4e65>", line 34, in return_paragraphs
    for paragraph in chapter['paragraphs']:
TypeError: string indices must be integers
ERROR:root:No chapters in PMC_1084337
Traceback (most recent call last):
  File "<ipython-input-11-ddacd84d4e65>", line 34, in return_paragraphs
    for paragraph in chapter['paragraphs']:
TypeError: string indices must be integers
ERROR:root:No chapters in PMC_4917243
Traceback (most recent call last):
  File "<ipython-input-11-ddacd84d4e65>", line 34, in return_paragraphs
    for paragraph in chapter['paragraphs']:
TypeError: string indices must be integers
ERROR:root:No chapters in PMC_3075230
Traceback (most recent call last):


Total journals: 1


In [ ]:
for pubs in list_of_pubs:
    for paper in pubs:
        actions = []
        cleaned = []
        datasetsent = []
        othersent = []
        
        for paragraph in paper['paragraphs']:
            if paragraph == {}:
                continue
            lines = (sent_detector.tokenize(paragraph.strip()))
            
            if len(lines) < 3:
                continue

            for i in range(len(lines)):
                words = nltk.word_tokenize(lines[i])
                lengths = [len(x) for x in words]
                average = sum(lengths) / len(lengths)
                if average < 4:
                    continue
                    
                twosentences = ''
                try:
                    twosentences = lines[i] + ' ' + lines[i-1]

                except:
                    twosentences = lines[i] + ' ' + lines[i+1]
                    
                datasetsent.append(twosentences)

            #cleaned.append(paragraph)

        for num, parag in enumerate(datasetsent):
            actions.append({
                "_index": "twosent",
                "_type": "twosentnorules",
                "_id": paper['_id'] + str(num),

                "_source" : {
                    "title" : paper['title'],
                    "content.chapter.sentpositive" : parag,
                    "paper_id":paper['_id']
                }})
            
        if len(actions) == 0:
            continue

#         res = helpers.bulk(es, actions)
#         print(res)

# 4 Using ES 

In [5]:
res = es.search(index="ir", body={"query": {"match_all": {}}}, size = 2)
print("Got %d Hits:" % res['hits']['total'])
for hit in res['hits']['hits']:
    print(hit['_source']['title'])
    print(hit['_source']['text'])
    file = hit['_source']['text']
    print('_'*10)

Got 19637 Hits:
Relationship of patients' age to histopathological features of breast tumours in  BRCA1  and  BRCA2  and mutation-negative breast cancer families
Distinct pathological features among BRCA1-associated tumours have been found when such tumours are compared with sporadic cancers; these features include high tumour grade, negativity for oestrogen receptor (ER), overexpression of p53, negativity for progesterone receptor (PR), and a higher proportion of medullary and atypical medullary carcinomas [1-3]. Recently, cDNA expression analyses have suggested a basal epithelial phenotype for BRCA1 tumors [4] and expression of cytokeratins 5/6 have been associated with BRCA1 tumours [5]. Among BRCA2-associated tumours, findings have been inconsistent, and in most cases no significant difference has been found between BRCA2-associated and sporadic cancers [1,2,6,7].In our previous report [8], we have shown, consistent with earlier studies, that BRCA1-associated cancers were diagnosed

In [23]:
x = 0
res = es.search(index="twosent", body={"query": {"match_all": {}}}, size = 100)
print("Got %d Hits:" % res['hits']['total'])
for hit in res['hits']['hits']:
    if 'data' in hit['_source']['content.chapter.sentpositive']:
        print(hit['_source']['content.chapter.sentpositive'])
        print('_'*50,x)
        x = x + 1

Got 2683879 Hits:
In Section 3, we describe the experimental procedure that we followed to collect the raw data on which the analyses are based. In Section 2, we review related work.
__________________________________________________ 0
In Section 4, we describe the statistical analysis of the data, the results of the analysis, and the cross-validation procedures and their outcomes. In Section 3, we describe the experimental procedure that we followed to collect the raw data on which the analyses are based.
__________________________________________________ 1
In Section 5, we offer discussion of the experiment and its results. In Section 4, we describe the statistical analysis of the data, the results of the analysis, and the cross-validation procedures and their outcomes.
__________________________________________________ 2
There are three main differences between this previous work and our work. They showed that on their subject programs, the final sufficient set determined a higher A

In [24]:
x = 0
res = es.search(index="devtwosentnew", body={"query": {"match_all": {}}}, size = 100)
print("Got %d Hits:" % res['hits']['total'])
for hit in res['hits']['hits']:
    if 'data' in hit['_source']['content.chapter.sentpositive']:
        print(hit['_source']['content.chapter.sentpositive'])
        print('_'*50,x)
        x = x + 1

Got 2261716 Hits:
Remember that this Is only one use of the data structurel it is accessed in at least four other places in the program.
__________________________________________________ 0
Also, the cost includes the space as well as time costs, so the calculation must take into account the (sometimes changing) sizes of the data structures extant during the algorithm.
__________________________________________________ 1
Thus, if we knew the properties of the program totally, it might be reasonable to assl~ne that the failure process would be purely random, reflecting the fluctuations of the input data stream.
__________________________________________________ 2
We attempted to mitigate this threat by sharing both the " raw " data collected during the interviews and the resulting report (this paper) with the participants of the study for their validation.
__________________________________________________ 3
Amazon would tell you that 'yes, you can use the Elastic MapReduce on healthcar

In [98]:
res = es.search(index = "ir", body = {"query": {"match": {"_id" : "PMC_4075597"}}}, size = 200)

print("Got %d Hits:" % res['hits']['total'])
for hit in res['hits']['hits']:
    print(hit['_id'], hit['_source']['title'])

Got 1 Hits:
PMC_4075597 Enhanced wound healing by topical administration of d-limonene in alloxan induced diabetic mice through reduction of pro-inflammatory markers and chemokine expression


In [106]:
res = es.search(index = "twosent", body = {"query": {"match": {"paper_id" : "PMC_4075597"}}}, size = 200)

print("Got %d Hits:" % res['hits']['total'])
for hit in res['hits']['hits']:
    print(hit['_id'], hit['_source'])

Got 9 Hits:
PMC_40755975 {'content.chapter.sentpositive': 'Limonene treated mice showed a significant decrease in wound size, the levels of GITR-expressing cells, and Th1 cytokines as well as substantial down regulation of mRNA expression of the inflammatory mediators compared with the vehicle-treated and diabetic mice. Furthermore, histopathological examination showed complete re-epithelisation, decreased inflammatory cells and presence of granulation tissue in the limonene treated mice.', 'title': 'Enhanced wound healing by topical administration of d-limonene in alloxan induced diabetic mice through reduction of pro-inflammatory markers and chemokine expression', 'paper_id': 'PMC_4075597'}
PMC_40755974 {'content.chapter.sentpositive': 'D-limonene, one of the major constituent in citrus essential oils is considered to have antioxidant, hypoglycemic and anti-inflammatory activities [2]. Diabetes can hinder the normal wound healing process by inducing long-term inflammation which can l

In [76]:
res = es.search(index = "ir", body = {"query": {"match": {"title" : "cancer"}}}, size = 5)

print("Got %d Hits:" % res['hits']['total'])
for hit in res['hits']['hits']:
    print(hit["_id"], hit['_source']['title'], hit['_score'])

Got 377 Hits:
PMC_1851393 Risk of second primary cancer in men with breast cancer 5.460385
PMC_4579663 An ecological measure of immune-cancer colocalization as a prognostic factor for breast cancer 4.976323
PMC_340959 A New Breast Cancer Model 4.824152
PMC_4940926 Systemic therapy for breast cancer and risk of subsequent contralateral breast cancer in the WECARE Study 4.7360334
PMC_3720249 Vav Proteins' Role in Skin Cancer 4.702413


In [77]:
res = es.search(index = "ir", body = {"query": {"match": {"title" : "genetic"}}}, size = 5)

print("Got %d Hits:" % res['hits']['total'])
for hit in res['hits']['hits']:
    print(hit['_id'], hit['_source']['title'])

Got 202 Hits:
PMC_2211537 The Genetic Structure of Pacific Islanders
PMC_4858160 Exploiting Genetic Interference for Antiviral Therapy
PMC_3185286 Find the weakest link. A comparison between demographic, genetic and demo-genetic metapopulation extinction times
conf_icse_WeimerNGF09 Automatically finding patches using genetic programming.
PMC_3341339 A Genetic Basis for Mechanosensory Traits in Humans


In [123]:
res = es.search(index = "ir", body = {"query": {'query_string' : {
                                        'query': '*onstruct*',
                                        'fields': ['text']}}}, size = 20)

print("Got %d Hits:" % res['hits']['total'])
for hit in res['hits']['hits']:
    print(hit['_id'], hit['_source']['title'])
    print('_'*100)

Got 8003 Hits:
PMC_2408596 Gene Structure Evolution of the Na + -Ca 2+  Exchanger ( NCX ) Family
____________________________________________________________________________________________________
PMC_4712596 Genomic and metagenomic analysis of microbes in a soil environment affected by the 2011 Great East Japan Earthquake tsunami
____________________________________________________________________________________________________
conf_trec_BreckBFHLM99 A Sys Called Qanda.
____________________________________________________________________________________________________
conf_trec_Westerveld06 Correlating Topic Rankings and Person Rankings to Find Experts.
____________________________________________________________________________________________________
conf_trec_MoulahiTMY14 IRIT at TREC 2014 Contextual Suggestion Track.
____________________________________________________________________________________________________
conf_trec_SutcliffeGMW03 Question Answering using the DLT Syst

# 5 Doc2vec Indexing

In [3]:
from pymongo import MongoClient
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import nltk
from nltk import tokenize
import config as cfg

sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
client = MongoClient('localhost:' + str(cfg.mongoDB_Port))
pub = client.pub.publications
db=client.pub

es = Elasticsearch([{'host': 'localhost', 'port': 9200}],timeout=30, max_retries=10, retry_on_timeout=True)

# es.cluster.health(wait_for_status='yellow', request_timeout=1)

In [7]:
file = ' An important difference between citing and co-citation networks, however, is size. Since papers in the biomedical sciences have a median of 30 articles in their reference lists, each citation event can be expected to add multiple papers to an article’s co-citation network (Fig 1D) but only one to its citing network. The latter are therefore highly vulnerable to finite number effects; in other words, for an article of interest with few citations, small changes in the citing network would have a disproportionate effect on how that article’s field was defined. We therefore chose to pursue co-citation networks as a way to describe an individual paper’s field.Having chosen our comparison group, we looked for a way to test how accurately co-citation networks represent an article’s field. One way to characterize groups of documents is to cluster them based on the frequency at which specific terms appear in a particular document relative to the frequency at which they appear in the entire corpus, a method known as term frequency–inverse document frequency (TF-IDF) . This is not a perfect approach, as it is possible to use entirely different words to describe similar concepts, but positive matches can be taken as a strong indication of similarity. Frequency of word occurrence can be converted into vectors so that the cosine of the angle between two vectors is a measurement of how alike the two documents are. To evaluate co-citation networks relative to journal of publication, which is often used as a proxy for field, we selected all papers published from 2002 to 2011 in each of six different journals that received exactly five citations during that same time frame.'

In [11]:
file = 'exposed to increasing volumes of mcf7 and mdamb231 breast cancer cell cm or incubated with control volumes of concentrated serumfree media for 48 h after the 48 h incubation time cells and debris were removed from the thp1 supernatant by centrifugation following incubation the recovered cell number was determined and cell viability established using trypan blue exclusion under the conditions in this study the treatment of thp1 cells with cm did not reduce the viability of the thp1 cellsthe cm protein levels were quantified using a protein assay biorad hercules ca usa and the results were compared with a standard curve of bsa concentrations the protein concentrations were used to normalize the amount of cm loaded onto the gel thus the total protein loaded was equivalent in each lane loading buffer 5 wv sds 0225 m triscl ph 68 50 wv glycerol 005 wv bromophenol blue 025 m dithiothreitol was added to the cm and the mixture was denatured by boiling for 5 minutes samples were loaded onto a 10 sds polyacrylamide gel with 10 μl of a widerange colored protein molecular weight marker invitrogen carlsbad ca usa also loaded onto the gel the proteins were subsequently transferred from the gel onto a nitrocellulose membrane amersham biosciences piscataway nj usa the membrane was incubated with mouse monoclonal antimmp2 '

In [ ]:
papernames=[]
###############################
file = open('data/allcorpus_papers.txt', 'r')
text = file.read()

sentences = tokenize.sent_tokenize(text)
count = 0
docLabels = []
actions = []

for i, sent in enumerate(sentences):
    try:
        neighbors = sentences[i + 1]
        neighbor_count = count + 1
        
    except:
        neighbors = sentences[i - 1]
        neighbor_count = count - 1

    docLabels.append(count)

    actions.append({
       "_index": "devtwosentnew",
       "_type": "devtwosentnorulesnew",
       "_id":count,

       "_source" : {
           "content.chapter.sentpositive" : sent,
           "content.chapter.sentnegtive": neighbors,
           "neighborcount":neighbor_count
       }})
    count = count + 1

print(len(sentences))
print(len(docLabels))

res = helpers.bulk(es, actions)
print(res)

# 5 Preparing doc2vec

In [112]:
from pymongo import MongoClient
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import math
import nltk
import string
import config as cfg
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

###############################

client = MongoClient('localhost:' + str(cfg.mongoDB_Port))
db = client.pub
pub = client.pub.publications
es = Elasticsearch(
    [{'host': 'localhost', 'port': 9200}], timeout=30, max_retries=10, retry_on_timeout=True
)
es.cluster.health(wait_for_status='yellow', request_timeout=1)

list_of_pubs=[]

def returnnames(mongo_string_search, db):
    # mongo_string_search = {"dblpkey": "{}".format(dblkey)}
    results = db.publications.find(mongo_string_search)
    chapters = list()
    chapter_nums = list()
    list_of_docs = list()
    # list_of_abstracts = list()
    merged_chapters = list()
    my_dict = {
        "_id": "",

    }
    for i, r in enumerate(results):
        # try:
        # list_of_sections = list()
        my_dict['_id'] = r['_id']
        list_of_docs.append((my_dict))

        my_dict = {
            "_id": "",

        }

    return list_of_docs

In [113]:
filter_conference = ["WWW", "ICSE", "VLDB", "PVLDB", "JCDL", "TREC",  "SIGIR", "ICWSM", "ECDL", "ESWC"]

for booktitle in filter_conference:
    mongo_string_search = {'$and': [{'booktitle': booktitle}, {'content.fulltext': {'$exists': True}}]}
    list_of_pubs.append(returnnames(mongo_string_search, db))

In [114]:
mongo_string_search = {'$and': [{'domain': 'Biomedical'}, {'content.fulltext': {'$exists': True}}]}
list_of_pubs.append(returnnames(mongo_string_search, db))

In [ ]:
papersText = []

for pubs in list_of_pubs:

    for cur in pubs:

        query = {"query":
            {"match": {
                "_id": {
                    "query": cur['_id'],
                    "operator": "and"
                }
            }
            }
        }

        res = es.search(index = "ir", body = query, size = 200)

        for doc in res['hits']['hits']:
            # sentence = doc["_source"]["text"].replace(',', ' ')
            fulltext = doc["_source"]["text"]

            fulltext = fulltext.translate(str.maketrans('', '', string.punctuation))
            
            papersText.append(fulltext.lower())
            
papersText = " ".join(papersText)

f1 = open("data/word2vecData.txt", "w")
f1.write(papersText)
f1.close()